In [5]:
'''
Developer: Ellie Khanh Truong
'''
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
import seaborn as sns
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from scipy.stats import loguniform
import os
from pathlib import Path

BASE_DIR = Path('__file__').resolve().parent.parent
file_path = os.path.join(BASE_DIR, 'data', 'processed', 'FEATURE_ENGINEERED_DATASET.csv')
df = pd.read_csv(file_path)
df = df.loc[:, ~df.columns.str.contains(r"^Unnamed")]
# make a file to save all the visualization to find the correlations between features
plot_dir = Path('/workspaces/Prenergyze/backend/plots')
plot_dir.mkdir(parents=True, exist_ok=True)

In [6]:
print(df.head())
print(df.describe())
print(df.info())
print(df.isna().sum())
print(df.columns)
print(df['relative_humidity_2m'].describe())

                  date     load  temperature_2m  apparent_temperature  \
0  2023-09-08 00:00:00  23779.0       25.789000             30.172592   
1  2023-09-08 01:00:00  23024.0       25.889000             29.865730   
2  2023-09-08 02:00:00  21600.0       25.539000             29.705338   
3  2023-09-08 03:00:00  19933.0       25.338999             29.572971   
4  2023-09-08 04:00:00  18047.0       24.639000             28.900917   

   relative_humidity_2m  vapour_pressure_deficit  pressure_msl  precipitation  \
0              81.04294                 0.628876        1012.1            0.0   
1              78.38454                 0.721350        1012.8            0.0   
2              82.50449                 0.571860        1013.6            0.0   
3              85.02667                 0.483634        1013.5            0.0   
4              89.19012                 0.334905        1013.2            0.0   

   cloud_cover  cloud_cover_low  ...  vapour_pressure_deficit_roll_mean_24

In [7]:
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by='date')
df = df.sort_values('date').reset_index(drop=True)
df['target'] = df['load'].shift(-1)  # move the value down 1 row

In [8]:
# drop the last line, because the target gna be NaN after shift
df = df.dropna(subset=['target']).reset_index(drop=True)
feature_cols = [c for c in df.columns if c not in {'date', 'load', 'target'}]
X = df[feature_cols].copy()
Y = df['target'].copy()
print("X shape", X.shape)
print("Y shape", Y.shape)
print("First features:", list(X.columns[:5]))
print("First 5 y values:", Y.head())

X shape (10942, 60)
Y shape (10942,)
First features: ['temperature_2m', 'apparent_temperature', 'relative_humidity_2m', 'vapour_pressure_deficit', 'pressure_msl']
First 5 y values: 0    23024.0
1    21600.0
2    19933.0
3    18047.0
4    16560.0
Name: target, dtype: float64


In [9]:
# 73 days of hourly data, split in 10 set of data
splitter = TimeSeriesSplit(
    n_splits=5, max_train_size=None, test_size=1752, gap=2)

for fold, (train_idx, test_idx) in enumerate(splitter.split(X)):
    print("TRAIN:", train_idx, "TEST:", test_idx)
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]

TRAIN: [   0    1    2 ... 2177 2178 2179] TEST: [2182 2183 2184 ... 3931 3932 3933]
TRAIN: [   0    1    2 ... 3929 3930 3931] TEST: [3934 3935 3936 ... 5683 5684 5685]
TRAIN: [   0    1    2 ... 5681 5682 5683] TEST: [5686 5687 5688 ... 7435 7436 7437]
TRAIN: [   0    1    2 ... 7433 7434 7435] TEST: [7438 7439 7440 ... 9187 9188 9189]
TRAIN: [   0    1    2 ... 9185 9186 9187] TEST: [ 9190  9191  9192 ... 10939 10940 10941]


In [10]:
plt.figure(figsize=(12, 5))
plt.plot(df['date'], df['load'], label='Load', linewidth=1)
plt.title("Electric Load over Time")
plt.xlabel("Date")
plt.ylabel("Load (NW)")
plt.legend()
out1 = plot_dir/'load_over_time.png'
plt.savefig(out1, dpi=150)
plt.close()

In [11]:
# weather panels(only if the columns exists)
for col in ['temperature_2m', 'relative_humidity_2m', 'precipitation']:
    if col not in df.columns:
        df[col] = pd.NA  # avoid KeyError
# share the same data scales
fig, ax = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
ax[0].plot(df['date'], df['temperature_2m'], color='orange')
ax[0].set_title("Temp over time")

ax[1].plot(df['date'], df['relative_humidity_2m'], color='pink')
ax[1].set_title("Humid over time")

ax[2].plot(df['date'], df['precipitation'], color='blue')
ax[2].set_title('Precipitation over time')

plt.xlabel("Date")
plt.tight_layout()
out2 = plot_dir / 'weather_panels.png'
fig.savefig(out2, dpi=150)
plt.close(fig)
print(f"Saved: {out2}")

Saved: \workspaces\Prenergyze\backend\plots\weather_panels.png


In [12]:
# heatmap the see the correlation with load
# for readability
num = df.select_dtypes('number')
if 'load' in num.columns and num.shape[1] >= 2:
    top_cols = num.corr(numeric_only=True)['load'].abs(
    ).sort_values(ascending=False).head(12).index
    plt.figure(figsize=(10, 8))
    sns.heatmap(num[top_cols].corr(), cmap='coolwarm',
                annot=True, vmin=-1, vmax=1)
    plt.title("Top correlation features")
    plt.tight_layout()
    plt.savefig(plot_dir/'corr_heatmap_top.png', dpi=300)
    plt.close()

In [13]:
pair_cols = ['load', 'temperature_2m', 'relative_humidity_2m', 'precipitation']
use = df[pair_cols].dropna()
if len(use) > 5000:  # make sure the data is not lagging
    use = use.sample(5000, random_state=42)
g = sns.pairplot(use, corner=True, plot_kws={'alpha': 0.3, 's': 8})
g.fig.suptitle("Scatter Matrix (sampled)", y=1.02)
g.savefig(plot_dir/'pairplot_sampled.png', dpi=300, bbox_inches='tight')
plt.close()

In [14]:
# visualize the temperature correlate with the target "load"
fig, axs = plt.subplots(1, 3, figsize=(18, 5))
sns.scatterplot(x='temperature_2m', y='load', data=df, alpha=0.3, ax=axs[0])
axs[0].set_title("Load vs Temperature (Current)")

sns.scatterplot(x='temperature_2m_lag_24h', y='load',
                data=df, alpha=0.3, ax=axs[1])
axs[1].set_title("Load vs Temperature (24h lag)")

sns.scatterplot(x='temperature_2m_roll_mean_24h',
                y='load', data=df, alpha=0.3, ax=axs[2])
axs[2].set_title("Load vs Temperature (24h rolling mean)")
out_path = plot_dir/"load_vs_temp_all.png"
fig.savefig(out_path, dpi=300)
plt.close(fig)

In [15]:
# visualize the humidity correlate with the target "load"
cols = [("relative_humidity_2m", "Load vs Humidity (Current)"),
        ("relative_humidity_2m_lag_24h", "Load vs Humidity (24h Lag)"),
        ("relative_humidity_2m_roll_mean_24h",
         "Load vs Humidity (24h Rolling Mean)")
        ]

In [16]:
# define the present before subsplot, only keep the data thats in the columns
present = [(c, t) for c, t in cols if c in df.columns]

if not present:
    raise ValueError("No humidity columns found in df plot")

fig, axs = plt.subplots(1, len(present), figsize=(6*len(present), 5))
if len(present) == 1:
    axs = [axs]

for ax, (col, title) in zip(axs, present):
    sns.scatterplot(data=df, x=col, y='load', alpha=0.3, s=12, ax=ax)
    ax.set_title(title)
    ax.set_xlabel("Relative Humidity (%)")
    ax.set_ylabel("Electric Load (MW)")
plt.tight_layout()
out_path2 = plot_dir/"load_vs_humidity_all.png"
fig.savefig(out_path2, dpi=300)
plt.close(fig)
# the correlation is weak and wobbly

In [17]:
# visualize for the precipitation
# using the roll 3h: total rainfall over the past 3 hours
fig, axs = plt.subplots(1, 2, figsize=(18, 5))
sns.scatterplot(x='precipitation', y='load', data=df, alpha=0.3, ax=axs[0])
axs[0].set_title("Current Precipitation")

sns.scatterplot(x='precipitation_roll_sum_3h',
                y='load', data=df, alpha=0.3, ax=axs[1])
axs[1].set_title("3h Rolling Sum")

plt.tight_layout()
out_path = plot_dir / "load_vs_precip_all.png"
fig.savefig(out_path, dpi=300)
plt.close(fig)

In [18]:
# shrink all the heavy tails data first
features_all = ['load_lag_1h', 'load_lag_2h', 'load_lag_3h', 'load_lag_24h', 'load_lag_168h',
                'temperature_2m', 'temperature_2m_lag_24h', 'temperature_2m_roll_mean_24h',
                'apparent_temperature', 'apparent_temperature_lag_24h',
                'relative_humidity_2m', 'pressure_msl',
                'precipitation', 'precipitation_roll_sum_3h',
                'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos'
                ]
features = [c for c in features_all if c in df.columns]
missing = sorted(set(features_all)-set(features))
if missing:
    print("Emergency I LOST MY FEATURE")
X = df[features].copy()
Y = df['target'].copy()

In [19]:
print(df[features].skew(numeric_only=True).sort_values(ascending=False))

precipitation                   8.993779
precipitation_roll_sum_3h       5.818093
load_lag_24h                    0.563134
load_lag_168h                   0.561288
load_lag_1h                     0.544743
load_lag_2h                     0.536500
load_lag_3h                     0.530103
month_sin                       0.179192
pressure_msl                    0.115543
hour_cos                        0.045430
dow_cos                         0.008210
dow_sin                         0.005889
month_cos                      -0.000873
hour_sin                       -0.060320
apparent_temperature_lag_24h   -0.464069
apparent_temperature           -0.491010
temperature_2m_lag_24h         -0.508439
temperature_2m                 -0.527043
relative_humidity_2m           -0.580456
temperature_2m_roll_mean_24h   -0.731006
dtype: float64


In [20]:
# u see the precipitation and roll sum 3h fucked up -> right skewed
skewed_candidates = ['precipitation', 'precipitation_roll_sum_3h']
skewed_features = [c for c in skewed_candidates if c in features]
normal_features = [c for c in features if c not in skewed_features]

In [21]:
# scaling the data
transformers = []
if skewed_features:
    skewed_pipe = Pipeline([
        ('log1p', FunctionTransformer(np.log1p, validate=False)),
        ('scale', StandardScaler())
    ])
    transformers.append(('skewed', skewed_pipe, skewed_features))
if normal_features:
    transformers.append(('normal', StandardScaler(), normal_features))

if not transformers:
    raise ValueError("We cooked! No usable features")
# build the pipeline for skewed features bro
pre = ColumnTransformer(transformers, remainder='drop')
# FINALLY TRAIN RIDGE
ridge_reg = Ridge(alpha=0.1, solver='cholesky')
ridge_pipe = Pipeline([
    ('prep', pre),
    ('ridge', ridge_reg)
])

In [22]:
ridge_search = RandomizedSearchCV(
    ridge_pipe, param_distributions={'ridge__alpha': loguniform(1e-3, 1e3)},
    n_iter=25, cv=splitter,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=1
)
ridge_search.fit(X, Y)
ridge_search_rmse = -ridge_search.best_score_
print(ridge_search_rmse)
print(ridge_search.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
1125.2904332148155
{'ridge__alpha': np.float64(0.0013289448722869186)}


In [23]:
# try Naive baseline model just in case


def rmse(a, b):
    return np.sqrt(mean_squared_error(a, b))


naive_rmse = []
for fold, (tr, te) in enumerate(splitter.split(X)):
    y_test = Y.iloc[te]
    y_pred_naive = df.loc[y_test.index, 'load']
    r = rmse(y_test, y_pred_naive)
    naive_rmse.append(r)
    print(f"Fold {fold+1} Naive RMSE: {r:.2f}")

print("Average Naive RMSE:", np.mean(naive_rmse))

Fold 1 Naive RMSE: 1293.22
Fold 2 Naive RMSE: 1693.30
Fold 3 Naive RMSE: 962.65
Fold 4 Naive RMSE: 1221.36
Fold 5 Naive RMSE: 1575.81
Average Naive RMSE: 1349.2671805980576
